<H1> Coursera-Capstone-Project- week 3

In [3]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libraries are imported')

Libraries are imported


<h2> Part one </h2>

In [14]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)

In [15]:
wiki_toronto = BeautifulSoup(result.content, 'lxml')

In [16]:
def clean_content(content):
    content = content.replace(' / ', ', ')
    content = content.replace('(', '')
    content = content.replace(')', '')
    return content

items = []
for item in wiki_toronto.table.find_all('td'):
    postal_code = item.p.b.text
    if item.p.span.text == "Not assigned":
        continue
    parts = [it for it in item.stripped_strings if it not in "()"]
    borough = parts[1]
    neighborhood = ' '.join(parts[2:])
    
    items.append((postal_code, clean_content(borough), clean_content(neighborhood)))

df_toronto = pd.DataFrame([it for it in items], columns=['PostalCode', 'Borough', 'Neighborhood'])
df_toronto.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District , Ryerson"


In [17]:
df_toronto.shape

(103, 3)

<h2> Part second</h2>

In [21]:
dfgeo = pd.read_csv(r"C:\Users\hp123\Downloads\Geospatial_Coordinates.csv")
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [24]:
df2 = pd.merge(df_toronto, dfgeo, on="PostalCode", how='left')

In [25]:
df2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


<h2> Part third</h2>

In [26]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [27]:
CLIENT_ID = 'YAGC11DLKOVZK4HIVHYJRBESZ12431LVRN3JCEJFYLTRT4NO' # Foursquare ID
CLIENT_SECRET = 'PE3MQKG1IFZYV0OZNP1BU1DFIXRV2FDPVEY5M5HCESVQH4B5' # Foursquare Secret
VERSION = '20180605' # API version

In [28]:
df2.set_index('PostalCode', inplace = True) 
neighborhood_latitude = df2.loc['M9V']['Latitude']
neighborhood_longitude = df2.loc['M9V']['Longitude']

In [29]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=YAGC11DLKOVZK4HIVHYJRBESZ12431LVRN3JCEJFYLTRT4NO&client_secret=PE3MQKG1IFZYV0OZNP1BU1DFIXRV2FDPVEY5M5HCESVQH4B5&v=20180605&ll=43.739416399999996,-79.5884369&radius=500&limit=100'

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60a74eeab9028412f18690b6'},
 'response': {'headerLocation': 'Rexdale',
  'headerFullLocation': 'Rexdale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 9,
  'suggestedBounds': {'ne': {'lat': 43.7439164045, 'lng': -79.58222007762089},
   'sw': {'lat': 43.734916395499994, 'lng': -79.59465372237912}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4be58dc4cf200f479154133c',
       'name': 'Shoppers Drug Mart',
       'location': {'address': '1530 Albion Rd',
        'crossStreet': 'Albion Mall',
        'lat': 43.741685,
        'lng': -79.584487,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.741685,
          'lng': -79.584487}],
        'distance': 405,
        'postalCode': 'M9V 1B4',

In [31]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
venues = results['response']['groups'][0]['items']

In [33]:
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues

<ipython-input-33-ea6da929ed3c>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups
0,e-0-4be58dc4cf200f479154133c-0,0,"[{'summary': 'This spot is popular', 'type': '...",4be58dc4cf200f479154133c,Shoppers Drug Mart,1530 Albion Rd,Albion Mall,43.741685,-79.584487,"[{'label': 'display', 'lat': 43.741685, 'lng':...",405,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (Albion Mall), Etobicoke ON M9...","[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",0,[]
1,e-0-4be70e26cf200f47e334153c-1,0,"[{'summary': 'This spot is popular', 'type': '...",4be70e26cf200f47e334153c,Popeyes Louisiana Kitchen,80-1530 Albion Rd,at Kipling Ave. (Albion Centre),43.741209,-79.584332,"[{'label': 'display', 'lat': 43.74120870478487...",385,M9V 1B4,CA,Etobicoke,ON,Canada,[80-1530 Albion Rd (at Kipling Ave. (Albion Ce...,"[{'id': '4d4ae6fc7a7b7dea34424761', 'name': 'F...",0,[]
2,e-0-4c633939e1621b8d48842553-2,0,"[{'summary': 'This spot is popular', 'type': '...",4c633939e1621b8d48842553,Subway,"6210 Finch Ave West, Store 103",at Albion Rd.,43.742645,-79.589643,"[{'label': 'display', 'lat': 43.74264512142215...",372,M9V 0A1,CA,Toronto,ON,Canada,"[6210 Finch Ave West, Store 103 (at Albion Rd....","[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",0,[]
3,e-0-4b04a05bf964a520c45522e3-3,0,"[{'summary': 'This spot is popular', 'type': '...",4b04a05bf964a520c45522e3,Sheriff's No Frills,1530 Albion Rd,at Finch Ave. W.,43.741696,-79.584379,"[{'label': 'display', 'lat': 43.741696, 'lng':...",413,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (at Finch Ave. W.), Etobicoke ...","[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",0,[]
4,e-0-4cd4738cdfb4a1cd4337535c-4,0,"[{'summary': 'This spot is popular', 'type': '...",4cd4738cdfb4a1cd4337535c,The Beer Store,1530 Albion Rd,Near Finch Ave. W.,43.741694,-79.584373,"[{'label': 'display', 'lat': 43.7416936, 'lng'...",413,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (Near Finch Ave. W.), Etobicok...","[{'id': '5370f356bcbc57f1066c94c2', 'name': 'B...",0,[]
5,e-0-4d8ba6910c4e41bdaaf7667f-5,0,"[{'summary': 'This spot is popular', 'type': '...",4d8ba6910c4e41bdaaf7667f,Pizza Pizza,"1530 Albion Road, Unit T25",NaN,43.741569,-79.584489,"[{'label': 'display', 'lat': 43.74156896801906...",397,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Road, Unit T25, Etobicoke ON M9V ...","[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",0,[]
6,e-0-5112b872e4b0c0a78d7bcd27-6,0,"[{'summary': 'This spot is popular', 'type': '...",5112b872e4b0c0a78d7bcd27,Sunny Foodmart,1620 Albion road,Albion Road and Finch Ave,43.741840,-79.590561,"[{'label': 'display', 'lat': 43.74184023292111...",319,NaN,CA,Toronto,ON,Canada,"[1620 Albion road (Albion Road and Finch Ave),...","[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",0,[]
7,e-0-4c1951d6834e2d7f2d3a2a80-7,0,"[{'summary': 'This spot is popular', 'type': '...",4c1951d6834e2d7f2d3a2a80,McDonald's,"1530 Albion Road, Unit F-1",NaN,43.741757,-79.584230,"[{'label': 'display', 'lat': 43.7417571, 'lng'...",427,M9V 1B4,CA,Toronto,ON,Canada,"[1530 Albion Road, Unit F-1, Toronto ON M9V 1B...","[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",0,[]
8,e-0-4ce971bff8653704e40db7c4-8,0,"[{'summary': 'This spot is popular', 'type': '...",4ce971bff8653704e40db7c4,LCBO,Albion Mall,1530 Albion Rd,43.741508,-79.584501,"[{'label': 'display', 'lat': 43.74150752194671...",392,M9V 1B4,CA,Toronto,ON,Canada,"[Albion Mall (1530 Albion Rd), Toronto ON M9V ...","[{'id': '4bf58dd8d48988d186941735', 'name': 'L...",0,[]


In [34]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Shoppers Drug Mart,Pharmacy,43.741685,-79.584487
1,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.741209,-79.584332
2,Subway,Sandwich Place,43.742645,-79.589643
3,Sheriff's No Frills,Grocery Store,43.741696,-79.584379
4,The Beer Store,Beer Store,43.741694,-79.584373


In [35]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
df2.head(10)

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M3B,North York,Don Mills North,43.745906,-79.352188
M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937


In [38]:
venues = getNearbyVenues(names=df2['Borough'],latitudes=df2['Latitude'],longitudes=df2['Longitude'])

North York
North York
Downtown Toronto
North York
Queen's Park
Etobicoke
Scarborough
North York
East York
Downtown Toronto
North York
Etobicoke
Scarborough
North York
East York
Downtown Toronto
York
Etobicoke
Scarborough
East Toronto
Downtown Toronto
York
Scarborough
East York
Downtown Toronto
Downtown Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
North York
North York
Scarborough
North York
North York
East Toronto
North York
York
North York
Scarborough
North York
North York
Central Toronto
Central Toronto
York
York
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Etobicoke
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Mississauga
Etobicoke
Scarborough
Central Toronto
Downtown Toronto
West Toronto
S

In [39]:
print(venues.shape)
venues.head()

(2136, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,North York,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,North York,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,North York,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,North York,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [40]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,114,114,114,114,114,114
Downtown Toronto,1190,1190,1190,1190,1190,1190
East Toronto,118,118,118,118,118,118
East York,75,75,75,75,75,75
Etobicoke,76,76,76,76,76,76
Mississauga,14,14,14,14,14,14
North York,246,246,246,246,246,246
Queen's Park,31,31,31,31,31,31
Scarborough,95,95,95,95,95,95


In [41]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 275 unique categories.


In [42]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
onehot.shape

(2136, 275)

In [44]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.008772,...,0.000000,0.000000,0.000000,0.000000,0.008772,0.000000,0.000000,0.000000,0.000000,0.008772
1,Downtown Toronto,0.00000,0.00084,0.000840,0.00084,0.00084,0.00084,0.001681,0.001681,0.010924,...,0.002521,0.000000,0.010924,0.000840,0.003361,0.000000,0.006723,0.000000,0.000000,0.005042
2,East Toronto,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.025424,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016949
3,East York,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.013333,0.000000,0.000000,0.000000,0.013333
4,Etobicoke,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.013158,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013158,0.000000,0.000000
5,Mississauga,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.071429,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.00813,0.00000,0.004065,0.00000,0.00000,0.00000,0.000000,0.000000,0.008130,...,0.000000,0.000000,0.000000,0.004065,0.008130,0.000000,0.000000,0.000000,0.012195,0.000000
7,Queen's Park,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258
8,Scarborough,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.010526,...,0.000000,0.000000,0.000000,0.000000,0.010526,0.000000,0.000000,0.000000,0.000000,0.000000
9,West Toronto,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.018987,0.000000,0.012658,0.000000,0.006329,0.000000,0.000000,0.012658


In [45]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1     Pizza Place  0.06
2  Sandwich Place  0.06
3            Park  0.05
4            Café  0.05


----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.11
1                 Café  0.05
2  Japanese Restaurant  0.03
3                Hotel  0.03
4           Restaurant  0.03


----East Toronto----
                venue  freq
0         Coffee Shop  0.06
1    Greek Restaurant  0.06
2  Italian Restaurant  0.04
3             Brewery  0.04
4              Bakery  0.03


----East York----
                 venue  freq
0          Coffee Shop  0.05
1                 Park  0.05
2                 Bank  0.05
3  Sporting Goods Shop  0.04
4         Burger Joint  0.04


----Etobicoke----
            venue  freq
0     Pizza Place  0.09
1  Sandwich Place  0.07
2     Coffee Shop  0.05
3          Bakery  0.04
4             Gym  0.04


----Mississauga----
                       venue  freq
0                Coffee S

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Pizza Place,Sandwich Place,Café,Park,Restaurant,Sushi Restaurant,Dessert Shop,Liquor Store,Trail
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Seafood Restaurant,Clothing Store
2,East Toronto,Greek Restaurant,Coffee Shop,Brewery,Italian Restaurant,Ice Cream Shop,Café,Fast Food Restaurant,Restaurant,American Restaurant,Bakery
3,East York,Coffee Shop,Bank,Park,Sporting Goods Shop,Pizza Place,Burger Joint,Middle Eastern Restaurant,Indian Restaurant,Intersection,Supermarket
4,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Bakery,Fast Food Restaurant,Pool,Pharmacy,Liquor Store,Grocery Store,Gym
5,Mississauga,Coffee Shop,Hotel,Middle Eastern Restaurant,American Restaurant,Sandwich Place,Gas Station,Fried Chicken Joint,Intersection,Burrito Place,Mediterranean Restaurant
6,North York,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Grocery Store,Shopping Mall,Bank,Park,Japanese Restaurant,Sandwich Place
7,Queen's Park,Coffee Shop,Sushi Restaurant,Yoga Studio,Bar,Beer Bar,Spa,Smoothie Shop,Burger Joint,Burrito Place,Sandwich Place
8,Scarborough,Intersection,Fast Food Restaurant,Coffee Shop,Bakery,Bank,Discount Store,Pharmacy,Pizza Place,Chinese Restaurant,Breakfast Spot
9,West Toronto,Café,Bar,Coffee Shop,Italian Restaurant,Bakery,Restaurant,Breakfast Spot,Park,Vegetarian / Vegan Restaurant,Diner


In [48]:
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 2, 2, 3, 0, 4, 2, 0])

In [49]:
merged = grouped

# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.008772,...,Coffee Shop,Pizza Place,Sandwich Place,Café,Park,Restaurant,Sushi Restaurant,Dessert Shop,Liquor Store,Trail
1,Downtown Toronto,0.0,0.00084,0.00084,0.00084,0.00084,0.00084,0.001681,0.001681,0.010924,...,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Seafood Restaurant,Clothing Store
2,East Toronto,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.025424,...,Greek Restaurant,Coffee Shop,Brewery,Italian Restaurant,Ice Cream Shop,Café,Fast Food Restaurant,Restaurant,American Restaurant,Bakery
3,East York,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,...,Coffee Shop,Bank,Park,Sporting Goods Shop,Pizza Place,Burger Joint,Middle Eastern Restaurant,Indian Restaurant,Intersection,Supermarket
4,Etobicoke,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.013158,...,Pizza Place,Sandwich Place,Coffee Shop,Bakery,Fast Food Restaurant,Pool,Pharmacy,Liquor Store,Grocery Store,Gym
